<a href="https://colab.research.google.com/github/OlegDenisenko/Data-analyst-Step1k/blob/main/4.%20A%D0%BDa%D0%BBu%D1%82u%D0%BA_%D0%B4a%D0%BD%D0%BD%D1%8B%D1%85_%D0%BE%D0%BF_St%D0%B5%D1%80uk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Очередной минипроект**

1) Имеются следующие данные о транзакциях в период с 01.12.2010 по 12.09.2011:

InvoiceNo – номер транзакции

StockCode – код товара

Description – описание товара

Quantity – количество единиц товара, добавленных в заказ

InvoiceDate – дата транзакции 

UnitPrice – цена за единицу товара

CustomerID – id клиента

Country – страна, где проживает клиент

Импортируйте данные data.csv.zip (в папке) с кодировкой ISO-8859-1. 
Запишите полученный датафрейм в retail, а названия колонок сохраните в переменную retail_columns.

In [1]:
import pandas as pd
import numpy as np

In [2]:
path = 'https://stepik.org/media/attachments/lesson/361623/data.csv.zip'

In [3]:
retail = pd.read_csv(path, encoding = 'ISO-8859-1')#

In [4]:
retail.shape

(541909, 8)

In [5]:
retail_columns = retail.columns

2) Проверьте, встречаются ли в данных повторяющиеся наблюдения, и в качестве ответа укажите их количество. Если они есть, то удалите их из retail.

In [6]:
start_shape = retail.shape[0]

In [7]:
retail.drop_duplicates(inplace=True)

In [8]:
start_shape - retail.shape[0]

5268

3) ⭐️Задание со звёздочкой!⭐️
Данные содержат в себе записи как и об успешных транзакциях, так и об отмененных. Если пользователь отменил заказ, в начале номера транзакции (InvoiceNo) ставится C (canceled). 
Сколько всего заказов отменили пользователи?

In [9]:
retail

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,12/9/2011 12:50,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12/9/2011 12:50,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12/9/2011 12:50,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12/9/2011 12:50,4.15,12680.0,France


In [10]:
retail.InvoiceNo.str.startswith('C').sum()

9251


4) Теперь отфильтруйте данные и оставьте в retail только те заказы, где Quantity > 0. В качестве ответа укажите число оставшихся строк.

In [11]:
#retail[retail['Quantity'] > 0].sort_values(by='Quantity')

In [12]:
retail.query('Quantity > 0').head(3)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom


5) 💀 Сложное задание! 💀
Посчитайте число заказов для каждого пользователя (CustomerID) из Германии (Germany). Оставьте только тех, кто совершил более N транзакций (InvoiceNo), где N – 80% процентиль. Запишите полученные id пользователей в germany_top (не весь датафрейм, только id). Идентификатор заказа – InvoiceNo. Для каждого заказа в данных может встречаться более 1 строки.

In [38]:
german_buyers = retail.query('Country == "Germany"')\
                .groupby('CustomerID', as_index = 'False')\
                .agg({'InvoiceNo' : pd.Series.nunique})\
                .rename(columns = {'InvoiceNo': 'num_orders'})

In [40]:
german_buyers = german_buyers.reset_index()#.InvoiceNo.sort_values()

In [41]:
percentile_80 = german_buyers.num_orders.sort_values().quantile(q=0.8)

In [42]:
germany_top = german_buyers.query('num_orders > @percentile_80')

6) ⭐️Задание со звёздочкой!⭐️
Используя объект с id пользователей (germany_top), полученный на предыдущем шаге, отфильтруйте наблюдения и оставьте в данных записи только по интересующим нас юзерам. Результирующий датафрейм запишите в top_retail_germany.

In [54]:
germany_top.reset_index(drop=True)

,CustomerID,num_orders
0,12471.0,49
1,12472.0,13
2,12474.0,30
3,12476.0,20
4,12481.0,11
5,12500.0,13
6,12569.0,35
7,12600.0,11
8,12619.0,13
9,12621.0,23


In [56]:
top_retail_germany = retail.query('CustomerID in @germany_top')

7) Вернемся к анализу полного датасета retail. Создайте колонку Revenue с суммой покупки, используя колонки Quantity и UnitPrice.

In [57]:
retail['Revenue'] = retail.Quantity * retail.UnitPrice

8) Для каждой транзакции (InvoiceNo), посчитайте финальную сумму заказа. В качестве ответа укажите топ-5 (InvoiceNo) по сумме заказа (через запятую с пробелом и в порядке убывания TotalRevenue)

In [61]:
sorted_revenues = retail.groupby('InvoiceNo', as_index = 'False')\
                  .agg({'Revenue': 'sum'})\
                  .sort_values('Revenue', ascending = False)

In [71]:
sorted_revenues.reset_index().InvoiceNo[:5].str.cat(sep = ', ')

'581483, 541431, 574941, 576365, 556444'